In [1]:
class MovieAPI():
    def __init__(self):
        self.index = []
        self.title_li = []
        self.link_li = []
        self.image_li = []
        self.year_li = []
        self.dir_li = []
        self.actor_li = []
        self.rating_li = []
    
    def get_movie(self, movie_list, dir_list, client_id, client_secret):
        import urllib.request
        import xmltodict
        import pandas as pd
        import time
        
        for i, movie in enumerate(movie_list):
            encText = urllib.parse.quote(movie)
            # url = "https://openapi.naver.com/v1/search/movie.json?query=" + encText # JSON 결과
            url = "https://openapi.naver.com/v1/search/movie.xml?query=" + encText # XML 결과
            time.sleep(0.5)
            request = urllib.request.Request(url)
            request.add_header("X-Naver-Client-Id",client_id)
            request.add_header("X-Naver-Client-Secret",client_secret)
            response = urllib.request.urlopen(request)
            rescode = response.getcode()
            if(rescode==200):
                response_body = response.read()
                dict_data = xmltodict.parse(response_body.decode('utf-8'), xml_attribs=False)
                search_li = dict_data['rss']['channel']
                if 'item' in search_li.keys():
                    search_li = search_li['item']
                    if type(search_li) == list:
                        for j in range(len(search_li)):
                            mv_title = search_li[j]['title'].replace('</b>','').replace('<b>','')
                            mv_rating = search_li[j]['userRating']
                            mv_dir = search_li[j]['director']
                            mv_dir = mv_dir.split('|')[:-1] if mv_dir != None else None
                            mv_actor = search_li[j]['actor']
                            mv_actor = mv_actor.split('|')[:-1] if mv_actor != None else None
                            if mv_title == movie and mv_rating != None and mv_dir != None and mv_actor != None:
                                if mv_rating != '0.00' and mv_dir[0] == dir_list[i]:
                                    self.index.append(i)
                                    self.title_li.append(mv_title)
                                    self.link_li.append(search_li[j]['link'])
                                    self.image_li.append(search_li[j]['image'])
                                    self.year_li.append(search_li[j]['pubDate'])
                                    self.dir_li.append(mv_dir)
                                    self.actor_li.append(mv_actor)
                                    self.rating_li.append(mv_rating)
                    elif type(search_li) == dict:
                        mv_title = search_li['title'].replace('</b>','').replace('<b>','')
                        mv_dir = search_li['director']
                        mv_dir = mv_dir.split('|')[:-1] if mv_dir != None else None
                        mv_actor = search_li['actor']
                        mv_actor = mv_actor.split('|')[:-1] if mv_actor != None else None

                        self.index.append(i)
                        self.title_li.append(mv_title)
                        self.link_li.append(search_li['link'])
                        self.image_li.append(search_li['image'])
                        self.year_li.append(search_li['pubDate'])
                        self.dir_li.append(mv_dir)
                        self.actor_li.append(mv_actor)
                        self.rating_li.append(search_li['userRating'])
            else:
                print("Error Code:" + rescode)
            if (i != 0 and i % 10 == 0) or i == (len(movie_list)-1):
                print(f'{i}번째 완료')
            if i != 0 and i % 500 == 0:    # 500번마다 savepoint로 저장해두기 
                pd.DataFrame([self.index, self.title_li, self.link_li, self.image_li, self.year_li, self.dir_li, self.actor_li, self.rating_li]).transpose().to_csv(f'savepoint{i}_df_cp949.csv', encoding='cp949')
                print('savepoint', i)
                
        movie_df = pd.DataFrame([self.index, self.title_li, self.link_li, self.image_li, self.year_li, self.dir_li, self.actor_li, self.rating_li]).transpose()
        movie_df.columns = ['순번', '영화명', '네이버URL', '이미지URL', '연도', '감독', '출연', '평점']
        movie_df = movie_df.set_index('순번')

        return movie_df

In [2]:
import pandas as pd

# 리스트 불러오기
movie_df = pd.read_csv('movie_resize.csv', encoding='utf-8') #, index_col=0)9
display(movie_df[:6])
movie_list = movie_df['영화명'].to_list()
dir_list = movie_df['감독'].to_list()
print('len =',len(movie_list), movie_list[:10])
print('len =',len(dir_list), dir_list[:10])

,영화명,감독,배급사,개봉일,영화형태,국적,전국스크린수,전국 매출액,전국 관객수,장르,등급,영화구분
0,명량,김한민,(주)씨제이이엔엠,2014-07-30,장편,한국,1587,135748398910,17613682,사극,15세관람가,일반영화
1,극한직업,이병헌,(주)씨제이이엔엠,2019-01-23,장편,한국,1978,139647979516,16264944,코미디,15세관람가,일반영화
2,신과함께-죄와 벌,김용화,롯데쇼핑㈜롯데엔터테인먼트,2017-12-20,장편,한국,1912,115698654137,14410754,판타지,12세관람가,일반영화
3,국제시장,윤제균,(주)씨제이이엔엠,2014-12-17,장편,한국,966,110828014630,14245998,드라마,12세관람가,일반영화
4,어벤져스: 엔드게임,안소니 루소,월트디즈니컴퍼니코리아 유한책임회사,2019-04-24,장편,미국,2835,122182694160,13934592,액션,12세관람가,일반영화
5,겨울왕국 2,제니퍼 리,월트디즈니컴퍼니코리아 유한책임회사,2019-11-21,장편,미국,2648,114810421450,13747792,애니메이션,전체관람가,일반영화


len = 4145 ['명량', '극한직업', '신과함께-죄와 벌', '국제시장', '어벤져스: 엔드게임', '겨울왕국 2', '아바타', '베테랑', '도둑들', '7번방의 선물']
len = 4145 ['김한민', '이병헌', '김용화', '윤제균', '안소니 루소', '제니퍼 리', '제임스 카메론', '류승완', '최동훈', '이환경']


In [3]:
client_id = "viO54DlhW87ESywUDwWb"
client_secret = "7ZHAscDgKs"

ma = MovieAPI()
naver_df = ma.get_movie(movie_list=movie_list[2000:], dir_list=dir_list, client_id=client_id, client_secret=client_secret)

naver_df.to_csv('naver_api_cp949_2000.csv', encoding='cp949')
# display(naver_df)


savepoint 0
10번째 완료
20번째 완료
30번째 완료
40번째 완료
50번째 완료
60번째 완료
70번째 완료
80번째 완료
90번째 완료
100번째 완료
110번째 완료
120번째 완료
130번째 완료
140번째 완료
150번째 완료
160번째 완료
170번째 완료
180번째 완료
190번째 완료
200번째 완료
210번째 완료
220번째 완료
230번째 완료
240번째 완료
250번째 완료
260번째 완료
270번째 완료
280번째 완료
290번째 완료
300번째 완료
310번째 완료
320번째 완료
330번째 완료
340번째 완료
350번째 완료
360번째 완료
370번째 완료
380번째 완료
390번째 완료
400번째 완료
410번째 완료
420번째 완료
430번째 완료
440번째 완료
450번째 완료
460번째 완료
470번째 완료
480번째 완료
490번째 완료
500번째 완료
savepoint 500
510번째 완료
520번째 완료
530번째 완료
540번째 완료
550번째 완료
560번째 완료
570번째 완료
580번째 완료
590번째 완료
600번째 완료
610번째 완료
620번째 완료
630번째 완료
640번째 완료
650번째 완료
660번째 완료
670번째 완료
680번째 완료
690번째 완료
700번째 완료
710번째 완료
720번째 완료
730번째 완료
740번째 완료
750번째 완료
760번째 완료
770번째 완료
780번째 완료
790번째 완료
800번째 완료
810번째 완료
820번째 완료
830번째 완료
840번째 완료
850번째 완료
860번째 완료
870번째 완료
880번째 완료
890번째 완료
900번째 완료
910번째 완료
920번째 완료
930번째 완료
940번째 완료
950번째 완료
960번째 완료
970번째 완료
980번째 완료
990번째 완료
1000번째 완료
savepoint 1000
1010번째 완료
1020번째 완료
1030번째 완료
1040번째 완료
1050번째 완료
1060번째 완료
1070번째 

c:\Anaconda3\lib\site-packages\pandas\core\indexes\base.py:6999: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


In [12]:
naver_df.index = naver_df.index - 2000
naver_df.index
naver_df.head()

,영화명,네이버URL,이미지URL,연도,감독,출연,평점
순번,,,,,,,
2000,오즈: 신기한 마법가루,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://ssl.pstatic.net/imgmovie/mdi/mit110/16...,2017,"[블라디미르 토로프킨, 다리나 슈미트, 표도르 드미트리에프]","[이장원, 김하영]",7.72
2003,모데카이,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://ssl.pstatic.net/imgmovie/mdi/mit110/11...,2015,[데이빗 코엡],"[조니 뎁, 이완 맥그리거, 기네스 팰트로, 올리비아 문, 폴 베타니, 제프 골드브럼]",5.46
2005,호텔 아르테미스,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://ssl.pstatic.net/imgmovie/mdi/mit110/16...,2018,[드류 피어스],"[소피아 부텔라, 데이브 바티스타, 스털링 K. 브라운, 조디 포스터]",6.49
2006,극장판 콩순이: 장난감나라 대모험,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://ssl.pstatic.net/imgmovie/mdi/mit110/20...,2020,[이선명],None,8.84
2007,쓰리 썸머 나잇,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://ssl.pstatic.net/imgmovie/mdi/mit110/12...,2014,[김상진],"[김동욱, 임원희, 손호준, 윤제문, 류현경]",6.65


In [13]:
naver_df.to_csv('naver_api_cp949_2000_4142.csv', encoding='cp949')

### ▼ 아래는 함수 확인용

In [57]:
import urllib.request
import xmltodict
import pandas as pd

index = []
title_li = []
link_li = []
image_li = []
year_li = []
dir_li = []
actor_li = []
rating_li = []

i = 0
movie = '명량'
encText = urllib.parse.quote(movie)
# url = "https://openapi.naver.com/v1/search/movie.json?query=" + encText # JSON 결과
url = "https://openapi.naver.com/v1/search/movie.xml?query=" + encText # XML 결과
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    dict_data = xmltodict.parse(response_body.decode('utf-8'), xml_attribs=False)
    search_li = dict_data['rss']['channel']['item']
    if type(search_li) == list:
        for j in range(len(search_li)):
            mv_title = search_li[j]['title'].replace('</b>','').replace('<b>','')
            mv_rating = search_li[j]['userRating']
            if mv_title == movie and mv_rating != '0.00':
                index.append(i)
                title_li.append(mv_title)
                link_li.append(search_li[j]['link'])
                image_li.append(search_li[j]['image'])
                dir_li.append(search_li[j]['director'].split('|'))
                actor_li.append(search_li[j]['actor'].split('|'))
                rating_li.append(mv_rating)
    elif type(search_li) == dict:
        mv_title = search_li['title'].replace('</b>','').replace('<b>','')
        index.append(i)
        title_li.append(mv_title)
        link_li.append(search_li['link'])
        image_li.append(search_li['image'])
        dir_li.append(search_li['director'].split('|'))
        actor_li.append(search_li['actor'].split('|'))
        rating_li.append(search_li['userRating'])
else:
    print("Error Code:" + rescode)

naver_df = pd.DataFrame([index, title_li, link_li, image_li, dir_li, actor_li, rating_li]).transpose()
naver_df.columns = ['순번', '영화명', '네이버URL', '이미지URL', '감독', '출연', '평점']

display(naver_df)

,순번,영화명,네이버URL,이미지URL,감독,출연,평점
0,0,겨울왕국,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://ssl.pstatic.net/imgmovie/mdi/mit110/10...,"[크리스 벅, 제니퍼 리, ]","[크리스틴 벨, 이디나 멘젤, ]",9.13


In [53]:
movie_list[1280:1289]

['롤러코스터',
 '스페이스 침스:우주선을 찾아서',
 '헨젤과 그레텔: 마녀 사냥꾼 3D',
 '상하이',
 '모털 엔진',
 '친구와 연인사이',
 '파이스토리 : 악당상어 소탕작전',
 '19곰 테드',
 '백 투 더 비기닝']

In [72]:
naver_df = ma.get_movie(movie_list=movie_list[1282:1285], dir_list=dir_list, client_id=client_id, client_secret=client_secret)

2번째 완료


c:\Anaconda3\lib\site-packages\pandas\core\indexes\base.py:6999: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


In [60]:
import urllib.request
import xmltodict
import pandas as pd

index = []
title_li = []
link_li = []
image_li = []
year_li = []
dir_li = []
actor_li = []
rating_li = []

movie = '헨젤과 그레텔: 마녀 사냥꾼 3D'
encText = urllib.parse.quote(movie)
# url = "https://openapi.naver.com/v1/search/movie.json?query=" + encText # JSON 결과
url = "https://openapi.naver.com/v1/search/movie.xml?query=" + encText # XML 결과
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    dict_data = xmltodict.parse(response_body.decode('utf-8'), xml_attribs=False)
    # search_li = dict_data['rss']['channel']['item']
    print(dict_data['rss']['channel'])
    # if search_li:
    #     if type(search_li) == list:
    #         for j in range(len(search_li)):
    #             mv_title = search_li[j]['title'].replace('</b>','').replace('<b>','')
    #             mv_rating = search_li[j]['userRating']
    #             mv_dir = search_li[j]['director']
    #             mv_dir = mv_dir.split('|')[:-1] if mv_dir != None else None
    #             mv_actor = search_li[j]['actor']
    #             mv_actor = mv_actor.split('|')[:-1] if mv_actor != None else None
    #             if mv_title == movie and mv_rating != None and mv_dir != None and mv_actor != None:
    #                 if mv_rating != '0.00' and mv_dir[0] == dir_list[i]:
    #                     index.append(i)
    #                     title_li.append(mv_title)
    #                     link_li.append(search_li[j]['link'])
    #                     image_li.append(search_li[j]['image'])
    #                     year_li.append(search_li[j]['pubDate'])
    #                     dir_li.append(mv_dir)
    #                     actor_li.append(mv_actor)
    #                     rating_li.append(mv_rating)
    #     elif type(search_li) == dict:
    #         mv_title = search_li['title'].replace('</b>','').replace('<b>','')
    #         mv_dir = search_li['director']
    #         mv_dir = mv_dir.split('|')[:-1] if mv_dir != None else None
    #         mv_actor = search_li['actor']
    #         mv_actor = mv_actor.split('|')[:-1] if mv_actor != None else None

    #         index.append(i)
    #         title_li.append(mv_title)
    #         link_li.append(search_li['link'])
    #         image_li.append(search_li['image'])
    #         year_li.append(search_li['pubDate'])
    #         dir_li.append(mv_dir)
    #         actor_li.append(mv_actor)
    #         rating_li.append(search_li['userRating'])



{'title': "Naver Open API - movie ::'헨젤과 그레텔: 마녀 사냥꾼 3D'", 'link': 'https://search.naver.com', 'description': 'Naver Search Result', 'lastBuildDate': 'Wed, 22 Mar 2023 11:10:11 +0900', 'total': '0', 'start': '1', 'display': '0'}
